## Nume studenti:
- Alexandra Manole
- Teodor Mihaescu

## Grupa: 382

# Data Set 2: Energy Efficency
### (Missing values: no)

In [22]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV

In [24]:
# Citeste datele
data = pd.read_csv('./Datasets/energy.csv').iloc[:768,:-2]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    float64
 6   X7      768 non-null    float64
 7   X8      768 non-null    float64
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(10)
memory usage: 60.1 KB


In [26]:
# Pregateste datele
X = data.loc[:, :'X8']
# y = data.loc[:, 'Y1':]
y1 = data.loc[:, 'Y1']
y2 = data.loc[:, 'Y2']

In [28]:
# Transformam problema dintr-una de regresie intr-una de clasificare, dand intervale de clasificare y-ilor
"""
Observam ca y1 si y2 se incadreaza intre urmatoarele valori:
y1: 6.01-43.1
y2: 10.9-48.03

Vom imparti datele in urmatoarele clase:
y1: 6-10, 10-20, 20-30, 30-40, 40-50
y2: 10-20, 20-30, 30-40, 40-50
"""
y1 = np.where(np.logical_and(y1>=6, y1<10), 0, y1)
y1 = np.where(np.logical_and(y1>=10, y1<20), 1, y1)
y1 = np.where(np.logical_and(y1>=20, y1<30), 2, y1)
y1 = np.where(np.logical_and(y1>=30, y1<40), 3, y1)
y1 = np.where(np.logical_and(y1>=40, y1<50), 4, y1)

y2 = np.where(np.logical_and(y2>=10, y2<20), 0, y2)
y2 = np.where(np.logical_and(y2>=20, y2<30), 1, y2)
y2 = np.where(np.logical_and(y2>=30, y2<40), 2, y2)
y2 = np.where(np.logical_and(y2>=40, y2<50), 3, y2)

y = np.stack((y1, y2), axis=1)

In [30]:
# Scaleaza datele
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
# y = min_max_scaler.fit_transform(y)

In [32]:
# Separarea setului de date
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, shuffle=True)

In [34]:
# Verifica daca sunt egal distribuite
print(Counter(y_train[:,0]))
print(Counter(y_test[:,0]))
print("\n")
print(Counter(y_train[:,1]))
print(Counter(y_test[:,1]))

Counter({1.0: 245, 3.0: 117, 2.0: 111, 4.0: 23, 0.0: 16})
Counter({1.0: 132, 2.0: 58, 3.0: 50, 4.0: 12, 0.0: 4})


Counter({0.0: 230, 2.0: 142, 1.0: 110, 3.0: 30})
Counter({0.0: 118, 2.0: 74, 1.0: 55, 3.0: 9})


## Model 1: LogisticRegression

In [36]:
# Model
"""
"""
model1 = MultiOutputClassifier(LogisticRegression(random_state=0, multi_class='multinomial'))
model1.fit(X_train, y_train)
y_hat = model1.predict(X_test)

In [38]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[1. 0.]
 [1. 1.]
 [1. 0.]
 [4. 3.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Failed:	[42 57]


In [40]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [42]:
# Custom scoreers for multiclass output
f1_scorer = metrics.make_scorer(metrics.f1_score, greater_is_better=False, average='micro')
accuracy_scorer = metrics.make_scorer(metrics.accuracy_score, greater_is_better=True)

In [44]:
# Cross Validation Y1
results_train = cross_validate(model1, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099945, 0.00099778, 0.        , 0.00099707, 0.00100136]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_f1_scorer': array([nan, nan, nan, nan, nan]),
 'train_f1_scorer': array([nan, nan, nan, nan, nan]),
 'test_accuracy': array([nan, nan, nan, nan, nan]),
 'train_accuracy': array([nan, nan, nan, nan, nan])}

In [46]:
# Cross Validation Y2
results_train = cross_validate(model1, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099516, 0.        , 0.00099683, 0.00099897, 0.00099897]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_f1_scorer': array([nan, nan, nan, nan, nan]),
 'train_f1_scorer': array([nan, nan, nan, nan, nan]),
 'test_accuracy': array([nan, nan, nan, nan, nan]),
 'train_accuracy': array([nan, nan, nan, nan, nan])}

In [48]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {
    'C' : np.linspace(1000, 2000, 10),# C = 1/lambda ; C mare => lambda mic
    'solver': ['saga', 'lbfgs']}
grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [50]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.9530165912518853
{'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 2}


## Model 2: KNeighborsClassifier

In [52]:
# Model
"""
"""
model2 = KNeighborsClassifier()
model2.fit(X_train, y_train)
y_hat = model2.predict(X_test)

In [54]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[1. 0.]
 [1. 1.]
 [1. 0.]
 [4. 3.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Failed:	[44 56]


In [56]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [58]:
# Cross Validation Y1
results_train = cross_validate(model2, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00299382, 0.00997305, 0.00297642, 0.00299311, 0.00199533]),
 'score_time': array([0.01896644, 0.01097107, 0.02194047, 0.01894736, 0.01595664]),
 'test_f1_scorer': array([-0.68181818, -0.78571429, -0.76623377, -0.67320261, -0.74509804]),
 'train_f1_scorer': array([-0.97882736, -0.96091205, -0.96091205, -0.95121951, -0.96910569]),
 'test_accuracy': array([0.68181818, 0.78571429, 0.76623377, 0.67320261, 0.74509804]),
 'train_accuracy': array([0.97882736, 0.96091205, 0.96091205, 0.95121951, 0.96910569])}

In [60]:
# Cross Validation Y2
results_train = cross_validate(model2, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00199342, 0.00299358, 0.00398993, 0.00199556, 0.0039885 ]),
 'score_time': array([0.0189507 , 0.01496434, 0.02393794, 0.01695251, 0.02393794]),
 'test_f1_scorer': array([-0.74675325, -0.67532468, -0.64935065, -0.60784314, -0.7124183 ]),
 'train_f1_scorer': array([-0.88599349, -0.88436482, -0.89576547, -0.88780488, -0.89593496]),
 'test_accuracy': array([0.74675325, 0.67532468, 0.64935065, 0.60784314, 0.7124183 ]),
 'train_accuracy': array([0.88599349, 0.88436482, 0.89576547, 0.88780488, 0.89593496])}

In [62]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'n_neighbors': list(range(1, 20)), 'p': [1, 2, 3]}
grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [63]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.8674208144796379
{'n_neighbors': 3, 'p': 1}


## Model 3: DecisionTreeClassifier

In [64]:
# Model
"""
"""
model3 = DecisionTreeClassifier(max_depth=2)
model3.fit(X_train, y_train)
y_hat = model3.predict(X_test)

In [65]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [2. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[1. 0.]
 [1. 1.]
 [1. 0.]
 [4. 3.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Failed:	[42 64]


In [66]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [67]:
# Cross Validation Y1
results_train = cross_validate(model3, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099611, 0.00299144, 0.0019908 , 0.00099707, 0.00099778]),
 'score_time': array([0.00199556, 0.00299215, 0.00199413, 0.00199461, 0.00499249]),
 'test_f1_scorer': array([-0.79220779, -0.87012987, -0.87662338, -0.74509804, -0.79084967]),
 'train_f1_scorer': array([-0.82084691, -0.80130293, -0.79967427, -0.83252033, -0.82113821]),
 'test_accuracy': array([0.79220779, 0.87012987, 0.87662338, 0.74509804, 0.79084967]),
 'train_accuracy': array([0.82084691, 0.80130293, 0.79967427, 0.83252033, 0.82113821])}

In [68]:
# Cross Validation Y2
results_train = cross_validate(model3, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099945, 0.00099683, 0.0009973 , 0.00200558, 0.00099778]),
 'score_time': array([0.00199509, 0.00199628, 0.00199533, 0.00298738, 0.00199461]),
 'test_f1_scorer': array([-0.66233766, -0.73376623, -0.73376623, -0.66666667, -0.62091503]),
 'train_f1_scorer': array([-0.79153094, -0.73452769, -0.73452769, -0.74471545, -0.75609756]),
 'test_accuracy': array([0.66233766, 0.73376623, 0.73376623, 0.66666667, 0.62091503]),
 'train_accuracy': array([0.79153094, 0.73452769, 0.73452769, 0.74471545, 0.75609756])}

In [69]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'max_depth': list(range(1,5)),
'min_samples_split': np.linspace(1,3,5),
'min_samples_leaf': np.linspace(0.1,0.5,5),
'max_features': ['sqrt', 'log2', None],
'criterion': ['gini', 'entropy']}
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [70]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.7225490196078432
{'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'min_samples_leaf': 0.2, 'min_samples_split': 1.0}


## Model 4: RandomForestClassifier

In [71]:
# Model
"""
"""
model4 = RandomForestClassifier(max_depth=2, random_state=0, ccp_alpha=0.2)
model4.fit(X_train, y_train)
y_hat = model4.predict(X_test)

In [72]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[1. 0.]
 [1. 1.]
 [1. 0.]
 [4. 3.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Failed:	[79 64]


In [73]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [74]:
# Cross Validation Y1
results_train = cross_validate(model4, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.24536824, 0.17654991, 0.26528883, 0.19997835, 0.16555929]),
 'score_time': array([0.01097059, 0.00897455, 0.01447678, 0.00897431, 0.01196647]),
 'test_f1_scorer': array([-0.62987013, -0.70779221, -0.70779221, -0.7124183 , -0.7124183 ]),
 'train_f1_scorer': array([-0.71009772, -0.69055375, -0.69055375, -0.68943089, -0.68943089]),
 'test_accuracy': array([0.62987013, 0.70779221, 0.70779221, 0.7124183 , 0.7124183 ]),
 'train_accuracy': array([0.71009772, 0.69055375, 0.69055375, 0.68943089, 0.68943089])}

In [75]:
# Cross Validation Y2
results_train = cross_validate(model4, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.21642137, 0.19749188, 0.1880064 , 0.19547749, 0.15059686]),
 'score_time': array([0.01296639, 0.01294541, 0.01396251, 0.01097107, 0.01097107]),
 'test_f1_scorer': array([-0.73376623, -0.73376623, -0.73376623, -0.73856209, -0.73202614]),
 'train_f1_scorer': array([-0.73452769, -0.73452769, -0.73452769, -0.73333333, -0.73495935]),
 'test_accuracy': array([0.73376623, 0.73376623, 0.73376623, 0.73856209, 0.73202614]),
 'train_accuracy': array([0.73452769, 0.73452769, 0.73452769, 0.73333333, 0.73495935])}

In [76]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'max_depth': list(range(5,10)),
'min_samples_split': list(range(1,4)),
'min_samples_leaf': list(range(1,5))}
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [79]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.9373303167420813
{'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2}


## Model 5: SVC (Support Vector Classification)

In [80]:
# Model
"""
Folosim metoda SVC pentru a clasifica observatiile. Atunci cand datele sunt in 4 sau mai multe dimensiuni SVC-ul este un hiper-plan. Acesta poate lucra cu valori extreme si din cazua ca permite clasificari eronate poate lucra si cu clasificari suprapuse.
"""
model5 = MultiOutputClassifier(SVC(C=2, gamma='auto'))
model5.fit(X_train, y_train)
y_hat = model5.predict(X_test)

In [81]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[1. 0.]
 [1. 1.]
 [1. 0.]
 [4. 3.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Failed:	[44 56]


In [82]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [83]:
# Cross Validation Y1
results_train = cross_validate(model5, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099134, 0.00199103, 0.00199509, 0.        , 0.00099921]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_f1_scorer': array([nan, nan, nan, nan, nan]),
 'train_f1_scorer': array([nan, nan, nan, nan, nan]),
 'test_accuracy': array([nan, nan, nan, nan, nan]),
 'train_accuracy': array([nan, nan, nan, nan, nan])}

In [84]:
# Cross Validation Y2
results_train = cross_validate(model5, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099802, 0.00099778, 0.00099778, 0.00099778, 0.00099778]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_f1_scorer': array([nan, nan, nan, nan, nan]),
 'train_f1_scorer': array([nan, nan, nan, nan, nan]),
 'test_accuracy': array([nan, nan, nan, nan, nan]),
 'train_accuracy': array([nan, nan, nan, nan, nan])}